In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [3]:
src_vocab = {}
EOS = '<eos>'
NIL = '<nil>'
entity_vocab = {}
id2wb = {}

In [4]:
import pickle
def read_data_from_pickle(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

src_lines = read_data_from_pickle('./data/seq2seq_wiki/src/src_list_1.pickle').decode('utf8').split('\n')
dest_lines = read_data_from_pickle('./data/seq2seq_wiki/dest_all/replace_entity_list_1.pickle').decode('utf8').split('\n')
entity_lines = read_data_from_pickle('./data/seq2seq_wiki/dest_entity/entity_list_1.pickle').decode('utf8').split('\n')

In [5]:
len(dest_lines)

4034

In [6]:
len(src_lines)

4034

In [7]:
len(entity_lines)

4034

In [8]:
for line in src_lines:
    lt = line.split()
    for w in lt:
        if w not in src_vocab:
            src_vocab[w] = len(src_vocab)

In [9]:
src_vocab[EOS] = len(src_vocab)
pv = len(src_vocab)

In [10]:
# entity_lines = open('./output.txt').read().split('\n')

In [11]:
for line in entity_lines:
    lt = line.split()
    for w in lt:
        if w not in entity_vocab:
            vocab_size = len(entity_vocab)
            entity_vocab[w] = vocab_size
            id2wb[vocab_size] = w
#     print(len(entity_vocab))

In [12]:
print(len(entity_vocab))
vocab_size = len(entity_vocab)
entity_vocab[EOS] = vocab_size
id2wb[vocab_size] = EOS

vocab_size = len(entity_vocab)
entity_vocab[NIL] = vocab_size
id2wb[vocab_size] = NIL

ev = len(entity_vocab)

9947


In [13]:
def mk_ct(gh, ht):
    alp = []
    s = 0.0
    for i in range(len(gh)):
        s += np.exp(ht.dot(gh[i]))
    ct = np.zeros(100)
    for i in range(len(gh)):
        alpi = np.exp(ht.dot(gh[i]))/s
        ct += alpi * gh[i]
    ct = Variable(np.array([ct]).astype(np.float32))
    return ct

In [14]:
class ATT(chainer.Chain):
    def __init__(self, pv, ev, k):
        super(ATT, self).__init__(
            embedx = L.EmbedID(pv, k),
            embedy = L.EmbedID(ev, k),
            H = L.LSTM(k, k),
            Wc1 = L.Linear(k, k),
            Wc2 = L.Linear(k, k),
            W = L.Linear(k, ev),
        )
        
    def __call__(self, pline, eline):
        gh = []
        for i in range(len(pline)):
            wid = src_vocab[pline[i]]
            x_k = self.embedx(Variable(np.array([wid], dtype=np.int32)))
            h = self.H(x_k)
            gh.append(np.copy(h.data[0]))
            
        x_k = self.embedx(Variable(np.array([src_vocab[EOS]], dtype=np.int32)))
        tx = Variable(np.array([entity_vocab[eline[0]]], dtype=np.int32))
        h = self.H(x_k)
        ct = mk_ct(gh, h.data[0])
        h2 = F.tanh(self.Wc1(ct) + self.Wc2(h))
        accum_loss = F.softmax_cross_entropy(self.W(h2), tx)
        
        for i in range(len(eline)):
            wid = entity_vocab[eline[i]]
            x_k = self.embedy(Variable(np.array([wid], dtype=np.int32)))
            next_wid = entity_vocab[EOS] if ( i == len(eline) - 1) else entity_vocab[eline[i+1]]
            tx = Variable(np.array([next_wid], dtype=np.int32))
            h = self.H(x_k)
            ct = mk_ct(gh, h.data)
            h2 = F.tanh(self.Wc1(ct) + self.Wc2(h))
            loss = F.softmax_cross_entropy(self.W(h2), tx)
            accum_loss += loss
            
        return accum_loss
    
    def reset_state(self):
        self.H.reset_state()

In [15]:
demb = 100
model = ATT(pv, ev, demb)
optimizer = optimizers.Adam()
optimizer.setup(model)

n_epoch = 20

print(len(src_lines))
import datetime

for epoch in range(n_epoch):
    sum_loss = 0
    
    for i in range(len(src_lines)-1):
        if i % 1000 == 0:
            print("{0}: Epoch {1} - Lines {2}...".format(datetime.datetime.today().strftime("%Y-%m-%d %H:%M:%S"), epoch, i))
        pln = src_lines[i].split()
        plnr = pln[::-1]
        eln = entity_lines[i].split()
        if len(eln) == 0:
            eln = [NIL]
#         print(eln)
        model.reset_state()
        model.zerograds()
        loss = model(plnr, eln)
        sum_loss += loss.data
        loss.backward()
        loss.unchain_backward()
        optimizer.update()
    print("{0} finished".format(epoch), flush=True)
    print("train loss: {0}".format(sum_loss), flush=True)
    
    if epoch == n_epoch-1:
        outfile = "attention-"+str(n_epoch)+".model"
        serializers.save_npz(outfile, model)

4034
2017-04-11 12:52:30: Epoch 0 - Lines 0...


2017-04-11 13:08:30: Epoch 0 - Lines 1000...


2017-04-11 13:23:49: Epoch 0 - Lines 2000...


2017-04-11 13:38:54: Epoch 0 - Lines 3000...


2017-04-11 13:53:54: Epoch 0 - Lines 4000...


...

2017-04-12 10:08:23: Epoch 19 - Lines 2000...


2017-04-12 10:23:57: Epoch 19 - Lines 3000...


2017-04-12 10:39:31: Epoch 19 - Lines 4000...


19 finished


train loss: 1939.6770641803741


In [30]:
def mt(model, src_line):
    lt = src_line.split()
    for i in range(len(lt)):
        wid = src_vocab[lt[i]]
        # print(lt[i])
        x_k = model.embedx(Variable(np.array([wid], dtype=np.int32)))
        h = model.H(x_k)
    x_k = model.embedx(Variable(np.array([src_vocab[EOS]], dtype=np.int32)))
    h = model.H(x_k)
    wid = np.argmax(F.softmax(model.W(h)).data[0])
    
    if wid in id2wb:
        print(id2wb[wid] + " / ")
    else:
        print(wid + " / ")
    
    loop = 0
    
    while (wid != entity_vocab[EOS]) and (loop <= 30):
        x_k = model.embedy(Variable(np.array([wid], dtype=np.int32)))
        h = model.H(x_k)
        wid = np.argmax(F.softmax(model.W(h)).data[0])
    
        if wid in id2wb:
            print(id2wb[wid] + " / ")
        else:   
            print(wid + " / ")
            
        loop += 1
    
    print()

In [31]:
for line in src_lines:
    print(line)
    mt(model, line)

ロートガウ ( Rodgau ) は 、 ドイツ連邦共和国 ヘッセン州 オッフェンバッハ郡 の 市 で ある 。 オッフェンバッハ・アム・マイン 南東 の ライン ＝ マイン 盆地 に 位置 し 、 オッフェンバッハ郡 で 最も 人口 の 多い 自治体 で ある 。 本市 は 、 ヘッセン州 の 地域 再編 に 伴っ て 、 1977年 に それ まで 独立 し た 自治体 で あっ た 5つ の 町村 が 合併 し て 成立 し た 広域 町村 ロートガウ が 1979年 に 市 と なっ た もの で ある 。 現在 の 市域 に 含ま れる 集落 の 歴史 は 、 8世紀 に まで 遡る 。
アメリカ合衆国 / 
アメリカ合衆国 / 
ドイツ / 
プロレスラー / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 

ロートガウ は 、 ドイツ で 最も 経済力 が 高い 地域 の 一つ で ある ライン ＝ マイン 大都市圏 の 一部 を なす 。 北緯 50 ° の 緯度 線 が 市内 ニーダー ＝ ローデン 市区 の ピューゾ 広場 を 通っ て いる 。 この街 は 、 オーバーライン 盆地 から 北 に 伸びる いわゆる ウンターマイン 盆地 （ マイン 川下 流域 盆地 ） に 位置 し て いる 。 これ を 、 いずれ も 近郊型 保養 地 と なっ て いる 中 低 山地 の シュペッサルト山地 、 タウヌス山地 、 フォーゲルスベルク山地 、 オーデンヴァルト および ベルクシュトラーセ が 取り囲ん で いる 。 この街 から わずか 数 km を 流れる マイン川 は 、 隣 の

<nil> / 
アメリカ合衆国 / 
ドイツ / 
イタリア / 
明治 / 
芸能事務所 / 
実業家 / 
シングル / 
アメリカ合衆国 / 
ドイツ / 
アメリカ合衆国 / 
ドイツ / 
指揮者 / 
弁護士 / 
南朝宋 / 
静岡県 / 
<eos> / 

気候 について は 、 この街 は ドイツ で 最も 温暖 で 、 最も 雨 の 少ない 地域 の 一つ に 属し て いる 。 1982年 から 2004年 まで の 平均 で 、 年間 平均 気温 は 10.5℃ 、 年間 降水量 は 639 . 1 mm で あっ た < ref > http % 3 A // www . dwd . de / ドイツ 気象庁 </ ref >。
<nil> / 
アメリカ合衆国 / 
ドイツ / 
自転車競技 / 
京都大学 / 
東京朝日新聞 / 
民主党 / 
民主党 / 
アルコール飲料 / 
ドイツ / 
日本 / 
シングル / 
日本 / 
リオデジャネイロオリンピック / 
イギリス / 
ナッシュビル / 
作曲家 / 
女優 / 
昭和 / 
韓国 / 
アメリカ合衆国 / 
ドイツ / 
自転車競技 / 
京都大学 / 
東京朝日新聞 / 
民主党 / 
ドイツ / 
<eos> / 

ロートガウ は 、 北 は ホイゼンシュタム および オーベルツハウゼン 、 東 は ハインブルク および ゼーリゲンシュタット （ 以上 、 オッフェンバッハ郡 ） 、 南東 は バーベンハウゼン および エッパーツハウゼン （ ともに ダルムシュタット＝ディーブルク郡 ） 、 南西 は レーダーマルク 、 西 は ディーツェンバッハ と 境 を 接し て いる 。
<nil> / 
アメリカ合衆国 / 
ドイツ / 
イタリア / 
第一次世界大戦 / 
パリ / 
第一次世界大戦 / 
端末 / 
ベトナム / 
テレビドラマ / 
NHK / 
NHK / 
スポーツ報知 / 
新潮社 / 
ニューヨーク州 / 
ジュークボックス・ミュージカル / 
第二次世界大戦 / 
ワートバーグ市 / 
サンフランシスコ / 
神奈川県 / 
弁護士 / 
長野県 / 
大正 / 
韓国 / 


第46回衆議院議員総選挙 / 
ヨーロッパ / 
ROCK / 
<eos> / 

ヘッセン州 の 地域 再編 に 伴い < ref > http % 3 A // www . hessenrecht . hessen . de / gesetze / 330 _ Allgemeines / 330 - 33 - NeugliederungsG - Offenbach / NeugliederungsG _ Offenbach . htm Gesetz zur Neugliederung des Landkreises Offenbach vom 26 .& nbsp ; Juni 1974 , GVBl . I S . 316 ; §& nbsp ; 6 Stadt Rodgau </ ref >、 1977年 に それ まで 独立 し た 自治体 で あっ た 上述 の 町村 （ 現在 の 市区 ） が 広域 町村 ロートガウ として 合併 し 、 1979年 9月19日 に 都市 権 を 付与 さ れ た 。 バッハガウ や キンツィヒガウ 同様 、 マインガウ に 属し た 昔 の 荘園 ロートガウ が 自治体 名 の 由来 で ある 。 合併 以前 の 自治体 は 、 それぞれ 何 百 年 も 遡る 歴史 を 有し て いる 。
<nil> / 
アメリカ合衆国 / 
ドイツ / 
イタリア / 
第一次世界大戦 / 
明治 / 
サンフランシスコ / 
サンフランシスコ / 
映画監督 / 
神奈川県 / 
弁護士 / 
長野県 / 
大正 / 
イタリア / 
大学院 / 
ロシア / 
大正 / 
イタリア / 
パリ / 
モスクワ / 
モスクワ / 
バンド / 
千葉県 / 
<eos> / 

2011年 12月31日 現在 の ロートガウ の 人口 は 45 , 096 人 （ 二次 居住者 2 , 241人 を 含む ） で 、 この うち 男性 は 22 , 163人 、 女性 は 22 , 933人 で ある 。 51 カ国 4 , 513人 ( 10 % ) の 外国人 が 含ま れる 。 住民 の 65 . 6 % が 10年 以上 ロートガウ に 居住 し て いる < ref name =" EWO "> ロートガウ 市 

ロサンゼルス / 
日本武道館 / 
昭和 / 
ケムニッツFC / 
ドイツ / 
菓子 / 
ドイツ / 
カリフォルニア州 / 
サンフランシスコ / 
民国 / 
東晋 / 
神奈川県 / 
オーケストラ / 
<eos> / 

5つ の プロテスタント 、 6つ の カトリック の 教会 および コミュニティセンター で は 定期的 に 祭事 が 行わ れ て いる 。 イスラム教 の 団体 は 、 ニーダー ＝ ローデン の 小 モスク で 集会 を 行っ て いる 。 ロートガウ 市民 の 25 . 5 % が プロテスタント 、 39 . 0 % が カトリック 、 35 . 5 % が その他 の 宗教 または 無 宗教 で ある 。
<nil> / 
アメリカ合衆国 / 
千葉県 / 
ベトナム / 
テレビドラマ / 
フジテレビ / 
バラエティ番組 / 
ウィーン / 
翻訳 / 
オランダ / 
アメリカ合衆国 / 
ドイツ / 
アメリカ合衆国 / 
ドイツ / 
アダルトゲーム / 
検事 / 
検事 / 
映画監督 / 
東宝映画 / 
東宝映画 / 
第47回衆議院議員総選挙 / 
フランス / 
地下鉄 / 
ミュージシャン / 
ミュージシャン / 
メリーランド州 / 
アメリカ合衆国 / 
ドイツ / 
軍人 / 
ナチス / 
イリノイ州 / 


メリーランド州 / 

ヴァイスキルヒェン の 東 郊外 に 、 1982年 から 、 カトリック の マインツ司教区 に 属す Internationalen Apostolischen Schönstatt - Bewegung の 会議 ・ 自己研鑽 センター が ある 。
アメリカ合衆国 / 
アメリカ合衆国 / 
ピアニスト / 
民主党 / 
レコーディング・エンジニア / 
ジョージア州 / 
バラエティ番組 / 
カンバーランド川 / 
第3.9世代移動通信システム / 
サンフランシスコ / 
イタリア / 
アラブ人 / 
中央大学 / 
アメリカ合衆国 / 
ファッションモデル / 
ナッシュビル / 
ピアニスト / 
第3.9世代移動通信システム / 
俳優 / 
ビルボード / 
日本テレビ / 
明治 / 
ドイツ / 
日本 / 
猿島郡 / 
猿島郡 / 
スパータ市 / 
明 / 
<nil> / 
DF / 
大都市圏 / 
民主党 / 

ヴァイスキルヒェン の 最初 の シナゴーグ は 、 1793年 に 保護 ユダヤ人 ゲダリー の 家 に 造ら れ た 。 その後 、 1881年 から 1882年 に 新しい 建物 が 建造 さ れ た 。 1938年 11月 （ 水晶の夜 ） に は 、 この 建物 は 売却 さ れ た 後 、 信徒 仲間 によって 住居 に 改築 さ れ て おり 、 破壊 を 免れ た 。 しかし 、 オッフェンバッハ・アム・マイン に 移さ れ て い た 宗教 関連 の 物品 は 破壊 さ れ 、 犠牲 と なっ た 。 ロートガウ の シナゴーグ だっ た 建物 は 、 所有 者 で ある 市 によって 改修 さ れ た 後 、 2010年 に ヴァイスキルヒェン 郷土 歴史 協会 によって 利用 さ れる こと と なっ た 。
<nil> / 
アメリカ合衆国 / 
ドイツ / 
イタリア / 
カンバーランド川 / 
明治 / 
芸能事務所 / 
実業家 / 
アメリカ / 
アメリカ / 
民主党 / 


...

<nil> / 
DF / 
作曲家 / 
<eos> / 

この 恒星ストリーム は 、 銀河系 中 の 球状星団 の 分布 を 分析 し た ドナルド・リンデンベル によって 1995年 に 提唱 さ れ た < ref name = LB 2 ></ ref >。 実際 の 構造 は 、 2002年 の Newberg ら < ref name = Discovery 1 > http % 3 A // adsabs . harvard . edu / abs / 2002 ApJ ... 569 .. 245 N The Ghost of Sagittarius and Lumps in the Halo of the Milky Way </ ref > や 2003年 の Majewski ら < ref name = Discover 21 > http % 3 A // adsabs . harvard . edu / abs / 2003 ApJ ... 599 . 1082 M A Two Micron All Sky Survey View of the Sagittarius Dwarf Galaxy . I . Morphology of the Sagittarius Core and Tidal Arms </ ref > によって 、 2 MASS や SDSS の データ から 確認 さ れ た 。 2006年 、 Belokurov ら は 、 いて座 ストリーム が 2つ の 分岐 を 持っ て いる こと を 発見 し た < ref name = Streams > http % 3 A // adsabs . harvard . edu / abs / 2006 ApJ ... 642 L . 137 B The Field of Streams : Sagittarius and Its Siblings </ ref >。
テレビアニメ / 
<nil> / 
千葉県 / 
千葉県 / 
<eos> / 

マゼラニックブリッジ ( Magellanic Bridge ) は 、 2つ の マゼラン雲 の 間 を 結ぶ 中性 水素 の 流れ で あり < ref > http % 3 A // www . publi

<nil> / 
アメリカ合衆国 / 
ドイツ / 
自転車競技 / 
京都大学 / 
京都大学 / 
カナダ / 
民主党 / 
イングランド / 
フランス / 
アイスホッケー / 
民主党 / 
第二次世界大戦 / 
文政 / 
ジョージア州 / 
第46回衆議院議員総選挙 / 
ジロ・デ・イタリア / 
ナッシュビル / 
ニューヨーク / 
日本 / 
神奈川県 / 
大阪府 / 
メリーランド州 / 
作曲家 / 
モデル / 
旧ユーゴスラビア国際戦犯法廷 / 
アメリカ合衆国 / 
ドイツ / 
イングランド / 
ドイツ / 
台湾 / 
京都大学 / 

イルマ・アギラール （） こと イルマ・ユージニア・アギラール・モラレス （< ref name =" data " />、 1957 % E 5% B 9% B4 1957年 7% E 6% 9 C % 8813 % E 6% 97% A5 7月13日 < ref name =" data " /> - ） は 、 メキシコ の 女子 プロレスラー 。 メキシコシティ 出身 < ref name =" data " />。 母 も 元 プロレスラー の イルマ・ゴンザレス < ref name =" rossy "></ ref >。
法学 / 
芥川龍之介 / 
民主党 / 
水原市 / 
水原市 / 
アメリカ合衆国大統領 / 
イギリス / 
ドイツ / 
日本 / 
スウェーデン / 
神奈川県 / 
大阪府 / 
ジョージア州 / 
民主党 / 
ドイツ / 
カリフォルニア州 / 
韓国 / 
アメリカ合衆国 / 
スタジオ・アルバム / 
スタジオ・アルバム / 
百家姓 / 
アダルトゲーム / 
第一次世界大戦 / 
テレビドラマ / 
ディン・トン / 
明治 / 
ドイツ / 
日本 / 
スウェーデン / 
神奈川県 / 
弁護士 / 
長野県 / 

1976 % E 5% B 9% B4 1976年 5% E 6% 9 C % 8815 % E 6% 97% A5 5月15日 デビュー < ref name =" data " />。
<nil> / 
ソウル / 
南北戦争 / 
アメリカ合衆国 / 


ドイツ / 
アメリカ合衆国 / 
ドイツ / 
アメリカ合衆国 / 
ドイツ / 
アメリカ合衆国 / 
ドイツ / 
京都大学 / 
オハイオ川 / 
ブラジル / 
第46回衆議院議員総選挙 / 
第46回衆議院議員総選挙 / 
オルト / 
ジェノヴァ / 
大学院 / 
ドイツ / 
アメリカ合衆国下院 / 
イタリア / 
正教会 / 
日本テレビ / 
サッカー / 
スイス / 
テレビ東京 / 
第46回衆議院議員総選挙 / 
アメリカ合衆国 / 
第一次世界大戦 / 
モスク / 
ドイツ / 

1979年 10月 、 全日本女子プロレス の 「 WWWA 世界タッグ リーグ戦 」 に 母 娘 で 初 来日 < ref name =" rossy " />。
<nil> / 
イングランド / 
大阪府 / 
ジョージア州 / 
ナッシュビル / 
トム・ハンクス / 
フェリー / 
フェリー / 
プロフェッショナルレフェリー / 
サッカー選手 / 
ニューヨーク州 / 
セリエA / 
大正 / 
イタリア / 
監督 / 
ロシア人 / 
フランス / 
花組 / 
アメリカ合衆国 / 
ドイツ / 
金属 / 
メリーランド州 / 
ニューヨーク州 / 
カナダ / 
バンド / 
マサチューセッツ工科大学 / 
シンガーソングライター / 
デトロイト・ライオンズ / 
国共内戦 / 
太魯閣号 / 
ドイツ / 
文政 / 

1983年 4月22日 、 ローラ・ゴンザレス を 破り UWA % E 4% B 8% 96% E 7% 95% 8 C % E 5% A5 % B 3% E 5% AD % 90% E 7% 8 E % 8 B % E 5% BA % A 7 UWA世界女子王座 を 奪取 < ref name =" owow "></ ref >。
アメリカ / 
シンガーソングライター / 


イラン・プロリーグ / 
兵庫県 / 
マンハッタン / 
カンバーランド川 / 
教授 / 
理学博士 / 
ダウンタウン / 
<nil> / 
アルゼンチン / 
ウェストバージニア州 / 
トム・クルーズ / 
<eos> / 

佐野 綾目 （ さ の あやめ 、 1936 % E 5% B 9% B4 1936年 - ） は 、 神戸市 生まれ 。 父 は 哲学者 佐野一彦 、 母 は ドイツ 出身 の ドイツ語 学者 、 翻訳家 佐野えんね 。
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 
アメリカ合衆国 / 

京都女子大学短期大学部 国語 専攻 卒業 。 美濃加茂市 読書 サークル 協議 会長 、 美濃加茂市 行政 相談員 、 美濃加茂市 教育委員長 を 歴任 し た 。 現在 、 まちづくり 活動 を 行っ て いる 。
<nil> / 
アメリカ合衆国 / 
ドイツ / 
イタリア / 
第一次世界大戦 / 
パリ / 
刺史 / 
松たか子 / 
#ロビンソン / 


ドイツ / 
サッカー選手 / 
テレビドラマ / 
ニューヨーク州 / 
オルタナティヴ・ロック / 
ヨーロッパ / 
アルゼンチン / 
サンフランシスコ / 
映画監督 / 
映画監督 / 
女優 / 
女優 / 
領主 / 
ベトナム / 
南北戦争 / 
アメリカ合衆国 / 
ドイツ / 
アメリカ合衆国 / 
スタジオ・アルバム / 
京都大学 / 
ウィーン / 
京都大学 / 
ベトナム / 

